In [1]:
import logging
import ricecooker.config
import chef
ricecooker.config.LOGGER.setLevel(logging.INFO)
ricecooker.config.LOGGER.addHandler(logging.StreamHandler())
#chef.construct_channel(language='hn').print_tree()

In [1]:
from bs4 import BeautifulSoup

from chef import LOGGER, PRADIGI_LANG_URL_MAP


hindi_start = PRADIGI_LANG_URL_MAP['hn']

from chef import PraDigiCrawler
    
crawler = PraDigiCrawler(start_page=hindi_start)


In [2]:
web_resource_tree = crawler.crawl()

Downloaded page http://www.prathamopenschool.org/hn/ title:POS
in on_lang_page http://www.prathamopenschool.org/hn/
found topic: Mathematics: गणित
found topic: English: अंग्रेजी
found topic: Health: स्वास्थ्य
found topic: Science: विज्ञान
found topic: Hospitality: अतिथी सत्कार
found topic: Construction: भवन-निर्माण
found topic: Automobile: वाहन
found topic: Electric: इलेक्ट्रिक
found topic: Beauty: ब्युटी
found topic: Healthcare: स्वास्थ्य सेवा
found topic: Std8: 8 वी कक्षा
found games page: Fun: मौज
found games page: Story: कहानियाँ
found topic: CRS104: खेल


skipping <a class="dropdown-toggle" data-toggle="dropdown" href="#" style="background: transparent;">
                                                कौशल <span class="caret"></span>
</a>


Downloaded page http://www.prathamopenschool.org/hn/Course/Mathematics title:Course
in on_topic_page http://www.prathamopenschool.org/hn/Course/Mathematics
  found subtopic: CRS17: ज्यामिती से परिचय
  found subtopic: CRS98: ब्याज
  found subtopic: CRS99: वर्ग संख्याएँ
  found subtopic: CRS27: गुणज और गुणनखंड
  found subtopic: CRS82: अलग संख्याएँ
  found subtopic: CRS41: मात्राएँ और संख्याएँ
  found subtopic: CRS24: प्रतिशत
  found subtopic: CRS21: बीजगणित से परिचय
  found subtopic: CRS40: दशमलव
Downloaded page http://www.prathamopenschool.org/hn/Course/English title:Course
in on_topic_page http://www.prathamopenschool.org/hn/Course/English
  found subtopic: CRS34: अंग्रेजी बोलो
  found subtopic: CRS36: अंग्रेजी ओके प्लीज
Downloaded page http://www.prathamopenschool.org/hn/Course/Health title:Course
in on_topic_page http://www.prathamopenschool.org/hn/Course/Health
  found subtopic: CRS28: आहार
Downloaded page http://www.prathamopenschool.org/hn/Course/Science title:Course
in on_topic_p

     in on_games_page http://www.prathamopenschool.org/hn/Fun


Downloaded page http://www.prathamopenschool.org/hn/Story title:Story


...

      in on_lesson_page http://www.prathamopenschool.org/hn/Lesson/LES524


Downloaded page http://www.prathamopenschool.org/hn/Lesson/LES525 title:POS-Lesson


      in on_lesson_page http://www.prathamopenschool.org/hn/Lesson/LES525


Downloaded page http://www.prathamopenschool.org/hn/Lesson/LES526 title:POS-Lesson


      in on_lesson_page http://www.prathamopenschool.org/hn/Lesson/LES526




################################################################################
# CRAWLER RECOMMENDATIONS BASED ON URLS ENCOUNTERED:
################################################################################

1. These URLs are very common and look like global navigation links:

2. These are common path fragments found in URLs paths, so could correspond to site struture:
  -  148 urls on site start with  /hn


################################################################################





In [4]:
# web_resource_tree

In [3]:
crawler.print_tree(web_resource_tree, print_depth=4)

     - path: /hn/  (lang_page) 
       children:
        - path: /hn/Course/Mathematics  (topic_page) 
          children:
           - path: /hn/Course/Mathematics/CRS17  (subtopic_page) 
             children:
              - path: /hn/Lesson/LES55  (lesson_page) 
              - path: /hn/Lesson/LES56  (lesson_page) 
              - path: /hn/Lesson/LES57  (lesson_page) 
              - path: /hn/Lesson/LES58  (lesson_page) 
              - path: /hn/Lesson/LES74  (lesson_page) 
              - path: /hn/Lesson/LES76  (lesson_page) 
           - path: /hn/Course/Mathematics/CRS98  (subtopic_page) 
             children:
              - path: /hn/Lesson/LES603  (lesson_page) 
           - path: /hn/Course/Mathematics/CRS99  (subtopic_page) 
             children:
              - path: /hn/Lesson/LES604  (lesson_page) 
           - path: /hn/Course/Mathematics/CRS27  (subtopic_page) 
             children:
              - path: /hn/Lesson/LES86  (lesson_page) 
              - path: /h